# 키워드 전처리/임베딩 과정

# imgKw 전처리 과정
## 결측치 확인

In [2]:
%pip install python-dotenv pymysql

In [12]:
from dotenv import load_dotenv
import os

# .env 파일의 환경 변수 로드
load_dotenv()

# 환경 변수에서 DB 암호 가져오기 (여기서는 변수명이 '1P_DB_PW'라고 가정)
db_password = os.getenv('1P_DB_PW')
print(db_password)

yttest1234


저장방식을 .csv가 아니라 .pkl (피클)을 쓴 이유

장점
빠르다text 파일을 사용하는 경우 필요한 부분들을 파싱해야 하지만 pickle은 이미 필요한 형태대로 저장이 되어 있기 때문에 훨씬 빠르다고 한다.
 

주의점
안전하지 않다[2]pickle module은 안전하지 않고 unpickle data만 신뢰할 수 있다고 한다.RCE(원격코드실행) 공격을 받을 수 있어, 다운 받은 pickle 파일을 사용할 때 매우 주의해야 한다.
 
검증 되지 않은 pickle data를 unpicking하게 될 때 임의의 코드가 실행될 수 있다고 한다.

## 전처리

In [43]:
import pymysql
import pandas as pd
import ast
import re

# DB 접속 설정
db_config = {
    'host': '121.128.172.79',
    'user': 'user3',
    'password': db_password,  # db_password 변수에 비밀번호를 지정하세요.
    'database': 'yttest_db',
    'port': 3306,
    'cursorclass': pymysql.cursors.DictCursor
}

# DB 연결 및 데이터 불러오기
conn = pymysql.connect(**db_config)
cursor = conn.cursor()
# video 테이블에서 thumbnailURL, imgKw, likeCount, viewCount, commentCount, catagoryID 컬럼을 가져옴
query = """SELECT v.thumbnailURL, t.imgKw, v.likeCount, v.viewCount, v.commentCount, v.categoryID, v.videoID
            FROM video v
            JOIN thumbnail t ON v.thumbnailURL = t.thumbnailURL;"""
cursor.execute(query)
data = cursor.fetchall()
conn.close()  # 데이터 불러온 후 연결 종료

# DataFrame 생성
df = pd.DataFrame(data)

# 결측치 처리: imgKw 컬럼의 결측값을 'unknown'으로 채워 넣기
df['imgKw'] = df['imgKw'].fillna('unknown')

# thumbnailURL 결측치 확인
if df['thumbnailURL'].isnull().sum() > 0:
    print("thumbnailURL 열에 결측치가 존재합니다. 추가 전처리가 필요합니다.")
else:
    print("thumbnailURL 열에 결측치가 없습니다.")

# 토큰 리스트 전처리 함수 정의
def clean_tokenized_keywords(token_str):
    if pd.isnull(token_str):
        return []
    token_str = token_str.strip()
    try:
        # 만약 token_str이 리스트 리터럴 형태라면 ast.literal_eval 시도
        if token_str.startswith('[') and token_str.endswith(']'):
            tokens = ast.literal_eval(token_str)
        else:
            # 그렇지 않으면 쉼표로 분리
            tokens = token_str.split(',')
    except Exception as e:
        # 변환 실패 시 쉼표로 분리
        tokens = token_str.split(',')
    
    cleaned_tokens = []
    for token in tokens:
        # 불필요한 기호 제거 및 소문자화
        token = token.strip("[]'\", ")
        token = re.sub(r'[^\w\s]', '', token)
        token = token.strip()
        if token:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

# 불용어 설정 (예: 'youtube', 'top', 'video', 'meter' 등)
stopwords = {'youtube', 'top', 'video', 'meter'}

def filter_stopwords(tokens, stopwords):
    return [t for t in tokens if t not in stopwords]

# imgKw 컬럼에 전처리 적용하여 cleaned_keywords 열 생성
df['cleaned_keywords'] = df['imgKw'].apply(clean_tokenized_keywords)
df['cleaned_keywords'] = df['cleaned_keywords'].apply(lambda tokens: filter_stopwords(tokens, stopwords))

# 결과 확인 (예시)
print(df[['imgKw', 'cleaned_keywords']].head())

# 저장 경로 (pkl 파일로 저장)
pkl_output_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\thumbnail_keywords_cleaned.pkl"

# DataFrame을 pickle 형식으로 저장 (모든 컬럼 포함: thumbnailURL, imgKw, cleaned_keywords, likeCount, viewCount, commentCount)
df.to_pickle(pkl_output_path)

print("전처리된 데이터가 .pkl 파일로 저장되었습니다.")

thumbnailURL 열에 결측치가 없습니다.
                                               imgKw  \
0  Sports venue, Baseball field, Baseball, Bat-an...   
1  Black hair, Mouth, Facial expression, News, Lo...   
2  Facial expression, News, Screenshot, Person, S...   
3  News, Lunch, Happiness, Screenshot, Comfort fo...   
4  Facial expression, Gesture, Screenshot, Cosmet...   

                                    cleaned_keywords  
0  [sports venue, baseball field, baseball, batan...  
1  [black hair, mouth, facial expression, news, l...  
2  [facial expression, news, screenshot, person, ...  
3  [news, lunch, happiness, screenshot, comfort f...  
4  [facial expression, gesture, screenshot, cosme...  
전처리된 데이터가 .pkl 파일로 저장되었습니다.


In [44]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101396 entries, 0 to 101395
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   thumbnailURL      101396 non-null  object
 1   imgKw             101396 non-null  object
 2   likeCount         101396 non-null  int64 
 3   viewCount         101396 non-null  int64 
 4   commentCount      101396 non-null  int64 
 5   categoryID        101396 non-null  object
 6   videoID           101396 non-null  object
 7   cleaned_keywords  101396 non-null  object
dtypes: int64(3), object(5)
memory usage: 6.2+ MB
None


## bert(kobert)모델로 임베딩하면서 토큰화 동시 진행

In [34]:
# pickle 파일에서 DataFrame 불러오기
import pickle

# pkl_file_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\thumbnail_keywords_cleaned.pkl"
# with open(pkl_file_path, 'rb') as f:
#     data = pickle.load(f)

# 위 방식보다 아래 방식으로 데이터를 불러오는게 더 코드가 간결함
data = pd.read_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\thumbnail_keywords_cleaned.pkl")  # 파일 경로 수정

    
print(data.info()) #["cleaned_keywords"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101396 entries, 0 to 101395
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   thumbnailURL      101396 non-null  object
 1   imgKw             101396 non-null  object
 2   likeCount         101396 non-null  int64 
 3   viewCount         101396 non-null  int64 
 4   commentCount      101396 non-null  int64 
 5   categoryID        101396 non-null  object
 6   cleaned_keywords  101396 non-null  object
dtypes: int64(3), object(4)
memory usage: 5.4+ MB
None


In [2]:
import pandas as pd
import numpy as np
import torch
import ast
from transformers import AutoTokenizer, AutoModel

c:\Users\YH\anaconda3\envs\model\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
# 모델과 토크나이저 로드 (bert-base-uncased 사용)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("사용 장치:", device)

사용 장치: cuda


In [36]:
# 토큰 파싱 함수 (중복된 함수 정의는 제거)
def parse_token_list(token_value):
    if isinstance(token_value, (list, np.ndarray)):
        return list(token_value)
    if pd.isnull(token_value):
        return []
    try:
        tokens = ast.literal_eval(token_value)
        return tokens if isinstance(tokens, list) else []
    except Exception:
        return token_value.split()

# BERT 임베딩 추출 함수
def get_bert_embedding_keywords(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    for key, value in inputs.items():
        inputs[key] = value.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        return cls_embedding[0].cpu().numpy()

# cleaned_keywords 열에 대해 임베딩 추출
all_keyword_embeddings = []

for idx, row in data.iterrows():
    keywords_list = parse_token_list(row['cleaned_keywords'])
    if len(keywords_list) == 0:
        embedding = np.zeros(model.config.hidden_size)
    else:
        combined_str = " ".join(keywords_list)
        embedding = get_bert_embedding_keywords(combined_str)
    all_keyword_embeddings.append(embedding)

X_keywords = np.vstack(all_keyword_embeddings)
print("cleaned_keywords의 BERT 임베딩 행렬 크기:", X_keywords.shape)

# DataFrame에 벡터 추가
data['bert_keyword_vector'] = list(all_keyword_embeddings)

# 아래는 .pkl방식으로 저장한다면 쓸 필요X. .csv 파일로 저장할때만 사용
# import json

# def vector_to_json(vector):
#     if isinstance(vector, np.ndarray):
#         vector = vector.tolist()
#     return json.dumps(vector)

# data['bert_keyword_vector_json'] = data['bert_keyword_vector'].apply(vector_to_json)

# 최종 결과를 pickle 파일로 저장 (또는 CSV도 함께 저장 가능)
data.to_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\merged_data_vector_0404.pkl")
# data.to_csv(r"/path/to/your/merged_data_vector_0401.csv", index=False, encoding="utf-8-sig")

cleaned_keywords의 BERT 임베딩 행렬 크기: (101396, 768)


### 모델 학습을 위한 데이터 셋 생성(likeCount, viewCount, commentCount 반영)

In [41]:
print(data["categoryID"])

0         5
1         9
2         1
3         1
4         6
         ..
101391    4
101392    3
101393    6
101394    1
101395    4
Name: categoryID, Length: 101396, dtype: object


In [38]:
import pandas as pd
import json
import numpy as np

# pickle 파일에서 데이터 불러오기
data = pd.read_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\merged_data_vector_0404.pkl")

# 만약 'bert_keyword_vector' 열이 이미 numpy array 형태라면 별도 변환은 필요하지 않습니다.
# 만약 JSON 문자열이라면 변환하는 함수를 사용하세요.
# def convert_embedding(x):
#     if isinstance(x, (list, np.ndarray)):
#         return np.array(x)
#     if pd.isnull(x):
#         return None
#     try:
#         # 우선 JSON 파싱 시도
#         return np.array(json.loads(x))
#     except Exception:
#         # JSON 파싱 실패 시 ast.literal_eval 시도
#         import ast
#         try:
#             return np.array(ast.literal_eval(x))
#         except Exception:
#             return None

# # 예시: 만약 'bert_keyword_vector_json' 열이 존재한다면 이를 사용하여 변환
# if 'bert_keyword_vector_json' in data.columns:
#     data['bert_keyword_vector'] = data['bert_keyword_vector_json'].apply(convert_embedding)

# 확인
print(data['bert_keyword_vector'].head())
print(data['bert_keyword_vector'].apply(lambda x: type(x)).head())

0    [-0.34264722, 0.033941608, -0.2915058, -0.0591...
1    [-0.17848471, -0.083290905, -0.15581135, -0.52...
2    [-0.118732065, 0.06530086, 0.013026365, -0.003...
3    [-0.2882179, -0.27976537, -0.12729499, -0.1137...
4    [-0.5027794, 0.083117984, -0.51912737, -0.2629...
Name: bert_keyword_vector, dtype: object
0    <class 'numpy.ndarray'>
1    <class 'numpy.ndarray'>
2    <class 'numpy.ndarray'>
3    <class 'numpy.ndarray'>
4    <class 'numpy.ndarray'>
Name: bert_keyword_vector, dtype: object


In [39]:
# 기존 코드: (N, 768)
X_keywords = np.vstack(data['bert_keyword_vector'].values)
print("X_keywords shape:", X_keywords.shape)

X_keywords shape: (101396, 768)


In [46]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101396 entries, 0 to 101395
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   thumbnailURL         101396 non-null  object
 1   imgKw                101396 non-null  object
 2   likeCount            101396 non-null  int64 
 3   viewCount            101396 non-null  int64 
 4   commentCount         101396 non-null  int64 
 5   categoryID           101396 non-null  object
 6   cleaned_keywords     101396 non-null  object
 7   bert_keyword_vector  101396 non-null  object
 8   videoID              101396 non-null  object
dtypes: int64(3), object(6)
memory usage: 7.0+ MB
None


In [47]:
import pandas as pd
import numpy as np
import json
import ast
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# # 각 동영상의 임베딩을 하나의 2차원 배열로 결합 (예: (N, 768))
# X_keywords = np.vstack(data['bert_keyword_vector'].values)
# print("X_keywords shape:", X_keywords.shape)

# 기존 코드: (N, 768)
X_keywords = np.vstack(data['bert_keyword_vector'].values)
print("X_keywords shape:", X_keywords.shape)

# 추가: PCA로 768차원을 512차원으로 축소
pca = PCA(n_components=512)
X_keywords_reduced = pca.fit_transform(X_keywords)
print("X_keywords_reduced shape:", X_keywords_reduced.shape)


# 예시 DataFrame: data
# data에는 videoID, categoryID, viewCount, likeCount, commentCount, bert_keyword_vector 열이 있다고 가정합니다.
# bert_keyword_vector 열은 이미 각 행에 대해 BERT 임베딩(예: 768차원 벡터)이 저장되어 있음

# 1) 임베딩 벡터 추출
# 예를 들어, data['bert_keyword_vector']가 리스트나 numpy 배열 형태라고 가정
X_keywords = np.vstack(data['bert_keyword_vector'].values)
print("X_keywords shape:", X_keywords.shape)

# 2) 회귀용 타겟: viewCount, likeCount, commentCount
y_view = data['viewCount'].values
y_like = data['likeCount'].values
y_comment = data['commentCount'].values
y_reg = np.stack([y_view, y_like, y_comment], axis=1)  # shape: (N, 3)

# 3) 분류용 타겟: categoryID를 재매핑 (원래 1~10 → 0~9)
data['categoryID'] = data['categoryID'].astype(int)
y_class = data['categoryID'].values - 1
print("Unique y_class:", np.unique(y_class))  # [0 1 2 3 4 5 6 7 8 9]가 출력되어야 함

# 4) videoID 보관 (후처리 및 결과 해석용)
video_ids = data['videoID'].values

# # 5) Train-Test split (예: 80% 학습, 20% 테스트)
# X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test, vid_train, vid_test = train_test_split(
#     X_keywords, y_reg, y_class, video_ids, test_size=0.2, random_state=42
# )

# 기존 코드에서 X_keywords 대신 X_keywords_reduced 사용

X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test, vid_train, vid_test = train_test_split(
    X_keywords_reduced, y_reg, y_class, video_ids, test_size=0.2, random_state=42
)

print("학습 데이터 X_train shape:", X_train.shape)
print("학습 데이터 y_reg_train shape:", y_reg_train.shape)
print("학습 데이터 y_class_train shape:", y_class_train.shape)
print("Unique y_class_train:", np.unique(y_class_train))  # [0 1 2 3 4 5 6 7 8 9]가 출력되어야 함

X_keywords shape: (101396, 768)
X_keywords_reduced shape: (101396, 512)
X_keywords shape: (101396, 768)
Unique y_class: [0 1 2 3 4 5 6 7 8 9]
학습 데이터 X_train shape: (81116, 512)
학습 데이터 y_reg_train shape: (81116, 3)
학습 데이터 y_class_train shape: (81116,)
Unique y_class_train: [0 1 2 3 4 5 6 7 8 9]


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import LabelEncoder
# Label encoding은 이미 실행했다고 가정합니다.
# 예시:
encoder = LabelEncoder()
data['categoryID_encoded'] = encoder.fit_transform(data['categoryID'])
y_class = data['categoryID_encoded'].values

# 데이터 정제: categoryID_encoded 열에 결측치가 있다면 제거
data = data.dropna(subset=['categoryID_encoded'])
data['categoryID_encoded'] = data['categoryID_encoded'].astype(int)

# 재매핑된 y_class를 사용합니다.
y_class = data['categoryID_encoded'].values
num_categories = len(np.unique(y_class))
print("새로운 카테고리 개수:", num_categories)

# 이후 train_test_split 과정에서, 정제된 y_class를 사용합니다.
from sklearn.model_selection import train_test_split

# 여기서 X_keywords 대신 PCA로 축소한 X_keywords_reduced를 사용합니다.
X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test, vid_train, vid_test = train_test_split(
    X_keywords_reduced, y_reg, y_class, video_ids, test_size=0.2, random_state=42
)

print("y_class_train unique values:", np.unique(y_class_train))

새로운 카테고리 개수: 10
y_class_train unique values: [0 1 2 3 4 5 6 7 8 9]


In [53]:
class MultiTaskModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_categories):
        super(MultiTaskModel, self).__init__()
        # 회귀를 위한 예시 레이어
        self.regressor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3)  # viewCount, likeCount, commentCount 예측
        )
        # 분류를 위한 예시 레이어
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_categories)
        )
    
    def forward(self, x):
        pred_reg = self.regressor(x)
        pred_class = self.classifier(x)
        return pred_reg, pred_class

In [58]:
# TensorDataset 구성
import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.FloatTensor(X_train)
y_reg_train_tensor = torch.FloatTensor(y_reg_train)
y_class_train_tensor = torch.LongTensor(y_class_train)
train_dataset = TensorDataset(X_train_tensor, y_reg_train_tensor, y_class_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 모델 초기화 (input_dim = X_train.shape[1] -> 512)
input_dim = X_train.shape[1]  # 512
hidden_dim = 256
model = MultiTaskModel(input_dim, hidden_dim, num_categories)
# 손실 함수 정의 (회귀와 분류 각각)
criterion_reg = nn.MSELoss()
criterion_class = nn.CrossEntropyLoss()

# optimizer 및 학습 관련 변수 정의)
num_epochs = 20
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y_reg, batch_y_class in train_loader:
        optimizer.zero_grad()
        pred_reg, pred_class = model(batch_X)
        loss_reg = criterion_reg(pred_reg, batch_y_reg)
        loss_class = criterion_class(pred_class, batch_y_class)
        loss = loss_reg + loss_class
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_X.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

import pickle

# 예를 들어, train_test_split 결과를 저장합니다.
dataset = {
    'X_train': X_train,
    'X_test': X_test,
    'y_reg_train': y_reg_train,
    'y_reg_test': y_reg_test,
    'y_class_train': y_class_train,
    'y_class_test': y_class_test,
    'vid_train': vid_train,
    'vid_test': vid_test
}

data.to_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\merged_data_set_0404.pkl")

print("데이터셋 저장 완료!")

Epoch 1/20, Loss: 35732317836.1339
Epoch 2/20, Loss: 35564997212.6994
Epoch 3/20, Loss: 35327751825.7839
Epoch 4/20, Loss: 35093194202.7366
Epoch 5/20, Loss: 34882191706.5804
Epoch 6/20, Loss: 34708237890.7583
Epoch 7/20, Loss: 34575299648.6659
Epoch 8/20, Loss: 34480485518.2729
Epoch 9/20, Loss: 34412088246.1818
Epoch 10/20, Loss: 34364316052.3560
Epoch 11/20, Loss: 34329448184.4825
Epoch 12/20, Loss: 34304156090.8084
Epoch 13/20, Loss: 34283066262.3254
Epoch 14/20, Loss: 34265366097.4999
Epoch 15/20, Loss: 34250544232.7026
Epoch 16/20, Loss: 34237599306.7840
Epoch 17/20, Loss: 34225364999.3850
Epoch 18/20, Loss: 34214922501.8827
Epoch 19/20, Loss: 34205105705.5452
Epoch 20/20, Loss: 34195631795.0637
데이터셋 저장 완료!


## 예측 모델 학습 및 평가

In [59]:
import pickle

data = pd.read_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\merged_data_set_0404.pkl")

X_train = dataset['X_train']
X_test = dataset['X_test']
y_reg_train = dataset['y_reg_train']
y_reg_test = dataset['y_reg_test']
y_class_train = dataset['y_class_train']
y_class_test = dataset['y_class_test']
vid_train = dataset['vid_train']
vid_test = dataset['vid_test']

print("데이터셋 불러오기 완료!")

데이터셋 불러오기 완료!


### TF-IDF 결과

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 인기 영상 필터링 (예: 상위 20% 영상만 선택)
popularity = data['viewCount'] + data['likeCount'] + data['commentCount']
threshold = np.percentile(popularity, 80)
# 슬라이스를 복사하여 사용 (SettingWithCopyWarning 해결)
popular_videos = data[popularity >= threshold].copy()

# NaN 값을 빈 문자열로 대체한 후, 각 동영상의 tokenized_keywords를 하나의 문자열로 변환
popular_videos['keywords_str'] = popular_videos['cleaned_keywords'].fillna("").apply(
    lambda x: " ".join(x) if isinstance(x, list) else x
)

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(popular_videos['keywords_str'])

# 각 단어의 평균 TF-IDF 점수 계산 (카테고리별로 나누어 할 수도 있음)
avg_tfidf = np.mean(tfidf_matrix.toarray(), axis=0)
keywords = tfidf_vectorizer.get_feature_names_out()
keyword_scores = list(zip(keywords, avg_tfidf))
# 점수가 높은 상위 20개 단어를 추천 키워드로 선택
recommended_keywords = sorted(keyword_scores, key=lambda x: x[1], reverse=True)[:20]
print("추천 키워드:", recommended_keywords)

추천 키워드: [('person', np.float64(0.05760085594020606)), ('food', np.float64(0.0562537281642235)), ('clothing', np.float64(0.04886552116566644)), ('screenshot', np.float64(0.040348741531132236)), ('facial', np.float64(0.034908806413900116)), ('expression', np.float64(0.03378454926732957)), ('news', np.float64(0.030894648014358293)), ('happiness', np.float64(0.028161077297271322)), ('film', np.float64(0.026103472383160204)), ('game', np.float64(0.025982370991353414)), ('advertising', np.float64(0.022682146418673513)), ('cuisine', np.float64(0.019846972956997926)), ('hair', np.float64(0.019562997563047625)), ('television', np.float64(0.019508443897107375)), ('cartoon', np.float64(0.019251372953768376)), ('device', np.float64(0.018994789071219407)), ('outerwear', np.float64(0.018806019001232407)), ('animation', np.float64(0.01869022203133106)), ('recipe', np.float64(0.01607365240222632)), ('show', np.float64(0.015992404881143948))]


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 우선, 기존에 인기 영상(popular_videos)이 설정되어 있다고 가정합니다.
# 예: popular_videos = data[popularity >= threshold].copy()

# 1. 카테고리 3(예: '여행')에 해당하는 인기 영상만 필터링합니다.
# 여기서는 원본 data의 'categoryID'가 사용된다고 가정합니다.
popular_videos_cat3 = popular_videos[popular_videos['categoryID'] == 2].copy()

# 2. NaN 값을 빈 문자열로 대체하고, tokenized_keywords를 하나의 문자열로 결합합니다.
popular_videos_cat3['keywords_str'] = popular_videos_cat3['cleaned_keywords'].fillna("").apply(
    lambda x: " ".join(x) if isinstance(x, list) else x
)

# 3. TF-IDF 벡터화: 새로 객체를 생성하여 카테고리 3 전용 TF-IDF 행렬을 만듭니다.
tfidf_vectorizer_cat3 = TfidfVectorizer(max_features=1000)
tfidf_matrix_cat3 = tfidf_vectorizer_cat3.fit_transform(popular_videos_cat3['keywords_str'])

# 4. 각 단어의 평균 TF-IDF 점수를 계산합니다.
avg_tfidf_cat3 = np.mean(tfidf_matrix_cat3.toarray(), axis=0)

# 5. 단어 목록과 TF-IDF 점수를 짝지어 정렬합니다.
keywords_cat3 = tfidf_vectorizer_cat3.get_feature_names_out()
keyword_scores_cat3 = list(zip(keywords_cat3, avg_tfidf_cat3))
recommended_keywords_cat3 = sorted(keyword_scores_cat3, key=lambda x: x[1], reverse=True)[:20]

print("카테고리 추천 키워드:")
for kw, score in recommended_keywords_cat3:
    print(f"{kw}: {score:.4f}")

카테고리 추천 키워드:
car: 0.1428
vehicle: 0.1057
kia: 0.0646
automotive: 0.0608
sport: 0.0578
luxury: 0.0574
utility: 0.0571
bmw: 0.0551
wheel: 0.0504
hyundai: 0.0478
device: 0.0440
tire: 0.0432
person: 0.0422
door: 0.0391
suv: 0.0351
clothing: 0.0345
toyota: 0.0331
compact: 0.0316
rover: 0.0315
display: 0.0314


In [64]:
# 1. 입력 특성 X 생성
# 각 동영상의 임베딩 벡터가 모두 잘 복원되어 있다고 가정합니다.
X = np.vstack(data['bert_keyword_vector'].dropna().values)  # shape: (N, 768)
print("X shape:", X.shape)

# 2. 타겟 생성: tokenized_keywords 열에서 candidate vocabulary 선정
from collections import Counter

all_keywords = []
# tokenized_keywords가 리스트형태로 저장되어 있다고 가정합니다.
for kw_list in data['cleaned_keywords']:
    if isinstance(kw_list, list):
        all_keywords.extend(kw_list)
    elif isinstance(kw_list, str):
        # 만약 문자열 형태라면, 공백 기준 분할
        all_keywords.extend(kw_list.split())

# 상위 500개 단어 선택
vocab_counter = Counter(all_keywords)
vocab = [kw for kw, cnt in vocab_counter.most_common(500)]
V = len(vocab)
print("Vocabulary 크기:", V)

# 함수: 각 동영상마다 multi-hot 벡터 생성
def create_target_vector(kw_list, vocab):
    target = np.zeros(len(vocab), dtype=np.float32)
    if isinstance(kw_list, list):
        for word in kw_list:
            if word in vocab:
                # vocab 내 단어의 인덱스를 찾아서 1로 설정
                idx = vocab.index(word)
                target[idx] = 1.0
    elif isinstance(kw_list, str):
        # 문자열인 경우 split 후 처리
        tokens = kw_list.split()
        for word in tokens:
            if word in vocab:
                idx = vocab.index(word)
                target[idx] = 1.0
    return target

# 각 동영상에 대해 multi-label 타겟 생성
# NaN 처리: NaN이면 빈 리스트로 간주
# 각 동영상에 대해 multi-label 타겟 생성
y_keywords = np.array([
    create_target_vector(kw_list, vocab) if isinstance(kw_list, (list, str))
    else np.zeros(V, dtype=np.float32)
    for kw_list in data['cleaned_keywords']
])
print("y_keywords shape:", y_keywords.shape)

X shape: (101396, 768)
Vocabulary 크기: 500
y_keywords shape: (101396, 500)


In [65]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

# train/validation 분할 (예: 80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y_keywords, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape, "X_val shape:", X_val.shape)

# 텐서 변환
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

X_train shape: (81116, 768) X_val shape: (20280, 768)


In [66]:
import torch.nn as nn
import torch.optim as optim

class KeywordRecommender(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KeywordRecommender, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim)  # 출력: 각 단어의 로짓 (logits)
        )

    def forward(self, x):
        return self.fc(x)

input_dim = X_train.shape[1]  # 예: 768
hidden_dim = 256
output_dim = V  # vocabulary 크기 (예: 500)
model = KeywordRecommender(input_dim, hidden_dim, output_dim)

In [67]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)  # outputs shape: (batch_size, V)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_X.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/20, Loss: 0.0433
Epoch 2/20, Loss: 0.0298
Epoch 3/20, Loss: 0.0273
Epoch 4/20, Loss: 0.0260
Epoch 5/20, Loss: 0.0252
Epoch 6/20, Loss: 0.0246
Epoch 7/20, Loss: 0.0241
Epoch 8/20, Loss: 0.0237
Epoch 9/20, Loss: 0.0234
Epoch 10/20, Loss: 0.0232
Epoch 11/20, Loss: 0.0230
Epoch 12/20, Loss: 0.0228
Epoch 13/20, Loss: 0.0226
Epoch 14/20, Loss: 0.0225
Epoch 15/20, Loss: 0.0224
Epoch 16/20, Loss: 0.0223
Epoch 17/20, Loss: 0.0221
Epoch 18/20, Loss: 0.0220
Epoch 19/20, Loss: 0.0220
Epoch 20/20, Loss: 0.0219


In [68]:
# 모델 및 관련 정보를 하나의 딕셔너리에 저장
save_dict = {
    'model_state_dict': model.state_dict(),
    'vocab': vocab,
    'input_dim': input_dim,    # 예: 768
    'hidden_dim': hidden_dim,  # 예: 256
    'output_dim': output_dim   # len(vocab)
}

save_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\keyword_recommender_full.pth"
torch.save(save_dict, save_path)
print("모델 및 관련 정보가 한 파일에 저장되었습니다.")

모델 및 관련 정보가 한 파일에 저장되었습니다.


In [69]:
import torch
import pickle
import torch.nn as nn

# 모델 구조 정의 (저장할 때와 동일하게)
class KeywordRecommender(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KeywordRecommender, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim)  # 출력: 각 단어의 로짓
        )

    def forward(self, x):
        return self.fc(x)

# 저장된 딕셔너리 불러오기
save_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\keyword_recommender_full.pth"
checkpoint = torch.load(save_path)

# 저장된 정보로 모델 초기화
input_dim = checkpoint['input_dim']
hidden_dim = checkpoint['hidden_dim']
output_dim = checkpoint['output_dim']
vocab = checkpoint['vocab']

model = KeywordRecommender(input_dim, hidden_dim, output_dim)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("모델 및 관련 정보 불러오기 완료!")

모델 및 관련 정보 불러오기 완료!


In [73]:
import torch
import pickle
import numpy as np

# 4. 추론 실행 (예: 카테고리 10의 추천 키워드 추출)
# 원본 데이터(data)에서 카테고리 10을 선택하고, 임베딩 벡터를 모읍니다.
selected_category = 5
subset = data[data['categoryID'] == selected_category].copy()
subset = subset.dropna(subset=['bert_keyword_vector'])
X_cat = np.vstack(subset['bert_keyword_vector'].values)
X_cat_tensor = torch.FloatTensor(X_cat)

with torch.no_grad():
    preds = model(X_cat_tensor)
    preds = torch.sigmoid(preds).cpu().numpy()

# 동영상별 예측 결과(각 행 벡터)를 평균하여 카테고리 전체의 추천 점수를 산출
avg_preds = np.mean(preds, axis=0)  # shape: (output_dim,)

# vocabulary와 점수를 짝지어 상위 추천 키워드 선정
keyword_scores = list(zip(vocab, avg_preds))
recommended_keywords = sorted(keyword_scores, key=lambda x: x[1], reverse=True)[:20]

print("카테고리", selected_category, "추천 키워드:")
for kw, score in recommended_keywords:
    print(f"{kw}: {score:.4f}")

카테고리 5 추천 키워드:
person: 0.7982
clothing: 0.5756
sports venue: 0.3134
sports: 0.2641
jersey: 0.1826
sports uniform: 0.1495
player: 0.1491
screenshot: 0.1368
stadium: 0.1156
advertising: 0.1102
shorts: 0.1056
baseball: 0.1055
soccer player: 0.1021
logo: 0.1020
hat: 0.0999
uniform: 0.0969
sports fan jersey: 0.0942
football player: 0.0941
football: 0.0883
cap: 0.0803
